# GRPA Coding

Goverment Perfromance and Results Act (GPRA) coding for intake data of recovery specialist for the Institure for Prevention and Recovery (IFPR) at RWJ Baranabas

In [1]:
#importing pandas
import pandas as pd
import numpy as np

In [2]:
# importing data and parsing datetime columns
df = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\Jan GPRA\GPRA_Coding_File.csv", parse_dates = ['ADMIT_DT','FORM_DTTM','FORM_UPDATE_DTTM'
])

#subsetting to include Peer Recovery Program episodes
df = df[df['EPISODE'] =='IFPR - STAR Program'].sort_values(by = ['FORM_DTTM','MRN']) # <- sorting df

In [3]:
# creating BatchID column
df['BatchID'] = '' 

In [4]:
df.rename(columns = {'CSN':'ClientID'},inplace = True)
df['ClientID'] = df['ClientID'].str.split(',').str[0]

In [5]:
df['PROVIDER_COUNTY'] = df['PROVIDER_COUNTY'].str.split(',').str[0]

In [6]:
#function for days columns that exceed 30 days
def exceed_thirty_days(x):
    if x > 30:
        return 30
    else:
        return x

## Creating GPRA 1

In [7]:
#Creating function to create column ClientDescripTreat
def Client_Descrip_Treat (x):
    if x == 'Treatment grant client':
        return 1
    else:
        return 0 
    
#Creating column ClientDescripTreat via applying function Client_Descrip_Treat
df['ClientDescripTreat'] = df['CLIENT_TYPE'].apply(lambda x: Client_Descrip_Treat (x))

In [8]:
#Creating function to create column ClientDescripRecov
def Client_Descrip_Recov (x):
    if x == 'Client in recovery grant':
        return 1
    else:
        return 0
    
#Creating column ClientDescripRecov via applying function Client_Descrip_Recov
df['ClientDescripRecov'] = df['CLIENT_TYPE'].apply(lambda x: Client_Descrip_Recov (x))

In [9]:
#Creating column GrantNo column
df['GrantNo'] = 'TI085743'

In [10]:
#Creating function for interview type
def interview_type_code(x):
    if x == 'Intake':
        return 1
    elif x == '6-month follow-up':
        return 2
    elif x == '3-month follow-up':
        return 4
    elif x == 'Discharge':
        return 5
    
#Creating new column via applying interview_type_code function to values in interview type column
df['InterviewType'] = df['INTERVIEW_TYPE'].apply(lambda x: interview_type_code(x))

In [11]:
#Creating function for conducted interview
def conducted_interview(x):
    if x == '6-month follow-up' or x == '3-month follow-up' or x == 'Discharge' or x == 'Intake':
        return 1
    else:
        return 0
    
# Creating a new column via applying the conducted_interview function to the values in the interview type column
df['ConductedInterview'] = df['INTERVIEW_TYPE'].apply(lambda x: conducted_interview(x))

In [12]:
#Renaming column to InterviewDate and changing data type to output in the format MM/DD/YYYY
df['InterviewDate'] = df['FORM_DTTM'].dt.strftime('%m/%d/%Y')

In [13]:
# Quality check
df['InterviewDate'] 

29      01/26/2023
696     01/27/2023
310     01/27/2023
586     01/27/2023
234     01/27/2023
           ...    
2457    09/03/2024
2461    09/04/2024
2460    09/04/2024
2466    09/06/2024
2465    09/06/2024
Name: InterviewDate, Length: 1299, dtype: object

In [14]:
#Creating InactFlag Column
df['MRN_CNT'] = df.groupby(['PATIENT'])['MRN'].transform('count') # <- will return count of MRN the length of df
df['MAX_ADMIT_DT'] = df.groupby(['PATIENT'])['ADMIT_DT'].transform('max') # <- will return MAX ADMIT_DT the length of df

# Function across multiple columns
def inactflag (row):
    if row['MRN_CNT'] == 1:
        return 0
    elif row['MRN_CNT'] > 1 and row['ADMIT_DT'] == row['MAX_ADMIT_DT']:
        return 0
    else:
        return 1
df['InactFlag'] = df.apply(inactflag, axis =1 ) # <- axis 1 will apply function across columns 

In [15]:
# Rolling Count to create IntakeSeqNum column
df['IntakeSeqNum'] = df.groupby('MRN')['ADMIT_DT'].cumcount() + 1 # <- rolling count +1 to start count at 1

In [16]:
# Assigning 0 for active grants
df['GrantInactFlag'] = 0

In [17]:
#Creating a function for Quarter
def year_qt (x):
    if x in [10,11,12]: # <- in kwarg that references a list 
        return 1
    elif x in [1,2,3]:
        return 2
    elif x in [4,5,6]:
        return 3
    elif x in [7,8,9]:
        return 4    

In [18]:
#Extracting year from ADMIT_DT to create column FFY
df['FFY'] = df['ADMIT_DT'].dt.year

#Extracting year from ADMIT_DT to create column Month
df['Month'] =  df['ADMIT_DT'].dt.month

#Extracting year from ADMIT_DT to create column Quarter
df['Quarter'] =  df['Month'].apply(lambda x: year_qt (x))

# Convering DOB to datetime format
df['DOB'] = pd.to_datetime(df['DOB'])

# Extracting year from DOB to create column BirthYear
df['BirthMonth'] = df['DOB'].dt.month

# Extracting year from DOB to create column BirthYear
df['BirthYear'] = df['DOB'].dt.year

# Calculating Age
df['Age'] = df['FFY'] - df['BirthYear']

# Creating function for Age Group
def age_group(x):
    if x >= 18 and x < 25:
        return 2
    elif x >= 25 and x < 35:
        return 3
    elif x >= 35 and x < 45:
        return 4
    elif  x >= 45 and x < 55:
        return 5
    elif x >=55 and x < 65:
        return 6
    elif x >= 65:
        return 7
    else:
        return -9
    
#Creating a new column via applying age_group function to values in age column
df['AgeGroup'] = df['Age'].apply(lambda x: age_group(x))

In [19]:
#Creating function for gender code 
def gender_code(x):
    if x == 'Male':
        return 1
    elif x == 'Female':
        return 2
    else:
        return 7

#Creating new column via applying gender_code function to values in Gender column
df['Gender'] = df['GENDER'].apply(lambda x: gender_code(x))

#Renaming column to create column GenderSpec
df.rename(columns = {'WHAT_DO_YOU_CONSIDER_YOURSELF_TO_BE_OTHER':'GenderSpec'}, inplace = True)

In [20]:
#Creating a function for HispanicLatino Column
def Hispanic_Latino (x):
    if x == 'Yes':
        return 1
    elif x == 'No (Skip to Question 3)':
        return 0
    elif  x == 'REFUSED (Skip to Question 3)':
        return -7
    else:
        return -9
    
#Creating new column HispanicLatino via applying Hispanic_Latino function to the values of the 'HISPANIC_LATIN_SPANISH_ORIGIN' column
df['HispanicLatino'] = df['HISPANIC_LATIN_SPANISH_ORIGIN'].apply(lambda x: Hispanic_Latino (x))

In [21]:
#Inputing 'NaN' for missing values in the WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF column
df['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] = df['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'].fillna('NaN')

#Creating substrings
df['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'] = df['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'].str.split(',').str[0]

#Creating a function for column EthnicCentranAmerican
def Central_American(x):
    if x == 'Central American':
        return 1
    elif x ==  'REFUSED':
        return -7
    elif x == 'NaN':
        return -1
    else:
        return 0
    
#Creating new column EthnicCentralAmerican via applying the Central_American function
df['EthnicCentralAmerican'] = df['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'].apply(lambda x:Central_American(x))

In [22]:
#Creating a function for column EthnicCuban
def Cuban(x):
    if x == 'Cuban':
        return 1
    elif x ==  'REFUSED':
        return -7
    elif x == 'NaN':
        return -1
    else:
        return 0
    
    
#Creating new column EthnicCuban via applying the Cuban function
df['EthnicCuban'] = df['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'].apply(lambda x:Cuban(x))

In [23]:
#Creating a function for column EthnicDominican
def DR(x):
    if x == 'Dominican':
        return 1
    elif x ==  'REFUSED':
        return -7
    elif x == 'NaN':
        return -1
    else:
        return 0
    
    
#Creating new column EthnicDominican via applying the DR function
df['EthnicDominican'] = df['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'].apply(lambda x:DR(x))

In [24]:
 #Creating a function for column EthnicMexican
def Mexican(x):
    if x == 'Mexican':
        return 1
    elif x ==  'REFUSED':
        return -7
    elif x == 'NaN':
        return -1
    else:
        return 0
    
    
#Creating new column EthnicMexican via applying the Mexican function
df['EthnicMexican'] = df['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'].apply(lambda x:Mexican(x))

In [25]:
#Creating a function for column EthnicPuertoRican
def PR(x):
    if x == 'Puerto Rican':
        return 1
    elif x ==  'REFUSED':
        return -7
    elif x == 'NaN':
        return -1
    else:
        return 0
    
    
#Creating new column EthnicPuertoRican via applying the PR function
df['EthnicPuertoRican'] = df['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'].apply(lambda x:PR(x))

In [26]:
#Creating a function for column EthnicSouthAmerican
def South_America(x):
    if x == 'South American':
        return 1
    elif x ==  'REFUSED':
        return -7
    elif x == 'NaN':
        return -1
    else:
        return 0
    
    
#Creating new column EthnicSouthAmerican via applying the South_America function
df['EthnicSouthAmerican'] = df['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'].apply(lambda x:South_America(x))

In [27]:
#Creating a function for column EthnicOther
def Ethnic_Other(x):
    if x == 'Other (SPECIFY)':
        return 1
    elif x ==  'REFUSED':
        return -7
    elif x == 'NaN':
        return -1
    else:
        return 0
    
    
#Creating new column EthnicSouthAmerican via applying the Ethnic_Other function
df['EthnicOther'] = df['WHAT_ETHNIC_GROUP_DO_YOU_CONSIDER_YOURSELF'].apply(lambda x:Ethnic_Other(x))

In [28]:
#Renaming column
df.rename(columns = {'ETHNIC_GROUP_OTHER':'EthnicOtherSpec'}, inplace = True)

In [29]:
#Fill null values with 'NaN' in the Race column
df['RACE'] = df['RACE'].fillna('NaN')

#Creating substrings 
df['RACE'] = df['RACE'].str.split(',').str[0]

#Creating function for column RaceBlack
def Race_Black(x):
    if x == 'Black or African American':
        return 1
    elif x == 'NaN':
        return -9
    elif x == 'REFUSED':
        return -7
    elif x == 'Other (SPECIFY), REFUSED':
        return -7
    else:
        return 0
    
#Creating new column RaceBlack via applying Race_Black function
df['RaceBlack'] = df['RACE'].apply(lambda x:Race_Black(x))

In [30]:
#Creating function for column RaceWhite
def Race_White(x):
    if x == 'White':
        return 1
    elif x == 'NaN':
        return -9
    elif x == 'REFUSED':
        return -7
    else:
        return 0
    
#Creating new column RaceWhite via applying Race_White function
df['RaceWhite'] = df['RACE'].apply(lambda x:Race_White(x))

In [31]:
#Creating function for column RaceAmericanIndian
def Race_Am_Indian(x):
    if x == 'American Indian':
        return 1
    elif x == 'NaN':
        return -9
    elif x == 'REFUSED':
        return -7
    else:
        return 0
    
#Creating new column RaceAmericanIndian via applying Race_Am_Indian function
df['RaceAmericanIndian'] = df['RACE'].apply(lambda x:Race_Am_Indian(x))

In [32]:
#Creating function for column RaceAlaskaNative
def Race_Alaska_Native(x):
    if x == 'Alaska Native':
        return 1
    elif x == 'NaN':
        return -9
    elif x == 'REFUSED':
        return -7
    else:
        return 0
    
#Creating new column RaceAlaskaNative via applying Race_Alaska_Native function
df['RaceAlaskaNative'] = df['RACE'].apply(lambda x:Race_Alaska_Native(x))

In [33]:
#Creating function for column RaceAsianIndian
def Race_Asian_Indian(x):
    if x == 'Asian Indian':
        return 1
    elif x == 'NaN':
        return -9
    elif x == 'REFUSED':
        return -7
    else:
        return 0
    
#Creating new column RaceAsianIndian via applying Race_Asian_Indian function
df['RaceAsianIndian'] = df['RACE'].apply(lambda x:Race_Asian_Indian(x))

In [34]:
#Creating function for column RaceChinese
def Race_Chinese(x):
    if x == 'Chinese':
        return 1
    elif x == 'NaN':
        return -9
    elif x == 'REFUSED':
        return -7
    else:
        return 0
    
#Creating new column RaceChinese via applying Race_Chinese function
df['RaceChinese'] = df['RACE'].apply(lambda x:Race_Chinese(x))

In [35]:
#Creating function for column RaceFilipino
def Race_Filipino(x):
    if x == 'Filipino':
        return 1
    elif x == 'NaN':
        return -9
    elif x == 'REFUSED':
        return -7
    else:
        return 0
    
#Creating new column RaceFilipino via applying Race_Filipino function
df['RaceFilipino'] = df['RACE'].apply(lambda x:Race_Filipino(x))

In [36]:
#Creating function for column RaceJapanese
def Race_Japanese(x):
    if x == 'Japanese':
        return 1
    elif x == 'NaN':
        return -9
    elif x == 'REFUSED':
        return -7
    else:
        return 0
    
#Creating new column RaceJapanese via applying Race_Japanese function
df['RaceJapanese'] = df['RACE'].apply(lambda x:Race_Japanese(x))

In [37]:
#Creating function for column RaceJapanese
def Race_Japanese(x):
    if x == 'Japanese':
        return 1
    elif x == 'NaN':
        return -9
    elif x == 'REFUSED':
        return -7
    else:
        return 0
    
#Creating new column RaceJapanese via applying Race_Japanese function
df['RaceJapanese'] = df['RACE'].apply(lambda x:Race_Japanese(x))

In [38]:
#Creating function for column RaceKorean
def Race_Korean(x):
    if x == 'Korean':
        return 1
    elif x == 'NaN':
        return -9
    elif x == 'REFUSED':
        return -7
    else:
        return 0
    
#Creating new column RaceKorean via applying Race_Korean function
df['RaceKorean'] = df['RACE'].apply(lambda x:Race_Korean(x))

In [39]:
#Creating function for column RaceVietnamese
def Race_Vietnamese(x):
    if x == 'Vietnamese':
        return 1
    elif x == 'NaN':
        return -9
    elif x == 'REFUSED':
        return -7
    else:
        return 0
    
#Creating new column RaceVietnamese via applying Race_Vietnamese function
df['RaceVietnamese'] = df['RACE'].apply(lambda x:Race_Vietnamese(x))

In [40]:
#Creating function for column RaceOtherAsian
def Race_Other_Asian(x):
    if x == 'Other Asian':
        return 1
    elif x == 'NaN':
        return -9
    elif x == 'REFUSED':
        return -7
    else:
        return 0
    
#Creating new column RaceOtherAsian via applying Race_Other_Asian function
df['RaceOtherAsian'] = df['RACE'].apply(lambda x:Race_Other_Asian(x))

In [41]:
#Creating function for column RaceNativeHawaiian
def Race_Hawaiian(x):
    if x == 'Hawaiian':
        return 1
    elif x == 'NaN':
        return -9
    elif x == 'REFUSED':
        return -7
    else:
        return 0
    
#Creating new column RaceNativeHawaiian via applying Race_Hawaiian function
df['RaceNativeHawaiian'] = df['RACE'].apply(lambda x:Race_Hawaiian(x))

In [42]:
#Creating function for column RaceGuamanianChamorro
def Race_Guamanian_Chamorro(x):
    if x == 'Guamanian Chamorro':
        return 1
    elif x == 'NaN':
        return -9
    elif x == 'REFUSED':
        return -7
    else:
        return 0
    
#Creating new column RaceGuamanianChamorro via applying Race_Guamanian_Chamorro function
df['RaceGuamanianChamorro'] = df['RACE'].apply(lambda x:Race_Guamanian_Chamorro(x))

In [43]:
#Creating function for column RaceSamoan
def Race_Samoan(x):
    if x == 'Samoan':
        return 1
    elif x == 'NaN':
        return -9
    elif x == 'REFUSED':
        return -7
    else:
        return 0
    
#Creating new column RaceSamoan via applying Race_Samoan function
df['RaceSamoan'] = df['RACE'].apply(lambda x:Race_Samoan(x))

In [44]:
#Creating function for column RaceOtherPacificIslander
def Race_Pacific_Islander(x):
    if x == 'Pacific Islander':
        return 1
    elif x == 'NaN':
        return -9
    elif x == 'REFUSED':
        return -7
    else:
        return 0
    
#Creating new column RaceOtherPacificIslander via applying Race_Pacific_Islander function
df['RaceOtherPacificIslander'] = df['RACE'].apply(lambda x:Race_Pacific_Islander(x))

In [45]:
#Creating function for column RaceOther
def Race_Other(x):
    if x == 'Other (SPECIFY)':
        return 1
    elif x == 'NaN':
        return -9
    elif x == 'REFUSED':
        return -7
    else:
        return 0
    
#Creating new column RaceOther via applying Race_Other function
df['RaceOther'] = df['RACE'].apply(lambda x:Race_Other(x))

In [46]:
#Renaming column 
df.rename(columns = {'RACE_OTHER':'RaceSpec'}, inplace = True)

#Renaming column to create column RaceSpec
df['RaceSpec'] = df['RaceSpec'].str.title()

In [47]:
# Filling missing in values in SPEAK_LANGUAGE_OTHER_THAN_ENGLISH column with 'NaN'
df['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] = df['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'].fillna('NaN')

#Creating function for LangNotEnglishAtHome
def Lang_Eng_Not_Spoken(row):
    if row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'No':
        return -1
    elif row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] == 'Yes':
        return 1
    elif row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] == 'No (Skip to Question 5)':
        return 0
    elif row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] == 'REFUSED (Skip to Question 5)':
        return -7
    elif row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] == 'NaN':
        return -9

def Lang_Eng_Not_Spoken_code_home(row):
    if row['InterviewType']==1:
        return Lang_Eng_Not_Spoken(row)
    else:
        return -1

df['LangNotEnglishAtHome'] = df.apply(Lang_Eng_Not_Spoken_code_home,axis=1)

#Creating function to create column LangNotEnglishSpoken
def Lang_Not_English_Spoken (row):
    if row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'No':
        return -1
    elif row['WHAT_OTHER_LANGUAGE'] == 'Spanish':
        return 1
    elif row['WHAT_OTHER_LANGUAGE'] == 'Other (SPECIFY)':
        return 2
    else:
        return -1
    
def Lang_Not_English_Spoken_code(row):
    if row['InterviewType']==1 and row['LangNotEnglishAtHome'] == 1:
        return Lang_Not_English_Spoken(row)
    else:
        return -1
    
df['LangNotEnglishSpoken'] = df.apply(Lang_Not_English_Spoken_code,axis=1)

In [48]:
df['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'].unique()

array([nan, 'Yes', 'No', 'REFUSED (Skip to Question 5)'], dtype=object)

In [49]:
#Renaming column to create column LangNotEnglishSpokenSpec
df.rename(columns = {'LANGUAGE_OTHER':'LangNotEnglishSpokenSpec'}, inplace = True)

In [50]:
#Creating LangNotSpanishSpokenSpec column
df['LangNotSpanishSpokenSpec'] = ''

In [51]:
#Creating function to create column LangNotSpanishAtHome
def Lang_Not_Spanish_Spoken_Home (row):
    if row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] == 'No (Skip to Question 5)':
        return -1
    elif row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'Yes' or row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'Other (SPECIFY)':
        return 1
    elif row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'No':
        return 0
    elif row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'REFUSED (Skip to Question 5)':
        return -7
    else:
        return -1
    
def Lang_Not_Spanish_Spoken_Home_code(row):
    if row['LangNotEnglishSpoken'] == 2:
        return -1
    elif row['LangNotEnglishAtHome'] == 0:
        return -1
    elif row['InterviewType']==1:
        return Lang_Not_Spanish_Spoken_Home(row)
    else:
        return -1
    
df['LangNotSpanishAtHome'] = df.apply(Lang_Not_Spanish_Spoken_Home_code,axis=1)  



#Creating function to create column LangNotSpanishSpoken
def Lang_Not_Spanish_Spoken (row):
    if row['SPEAK_LANGUAGE_OTHER_THAN_ENGLISH'] == 'No (Skip to Question 5)':
        return -1
    elif row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'Yes':
        return 1
    elif row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'Other (SPECIFY)':
        return 2
    elif row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'REFUSED (Skip to Question 5)':
        return -7
    elif row['SPEAK_LANGUAGE_OTHER_THAN_SPANISH'] == 'No':
        return -1
    else:
        return -1
    
def Lang_Not_Spanish_Spoken_code(row):
    if row['LangNotEnglishAtHome'] == 0:
        return -1
    elif row['LangNotEnglishSpoken'] == 2:
        return -1
    elif row['InterviewType']==1 and row['LangNotSpanishAtHome']==1:
        return Lang_Not_Spanish_Spoken(row)
    else:
        return -1
    
df['LangNotSpanishSpoken'] = df.apply(Lang_Not_Spanish_Spoken_code,axis=1) 

In [52]:
#filling missing values with 'NaN'
df['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] = df['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'].fillna('NaN')

#Creating substrings
df['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'] = df['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'].str.split(',').str[0]

#Creating function for column SexIdentHeterosexual
def Straight (x):
    if x == 'Straight or Heterosexual':
        return 1
    elif x == 'REFUSED':
        return -7
    elif x == 'NaN':
        return -9
    else:
        return 0
    
#Creating new column SexIdentHeterosexual via applying function Straight
df['SexIdentHeterosexual'] = df['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'].apply(lambda x:Straight (x))

In [53]:
#Creating function for column SexIdentHomoSexual
def Homosexual (x):
    if x == 'Homosexual (Gay or Lesbian)':
        return 1
    elif x == 'REFUSED':
        return -7
    elif x == 'NaN':
        return -9
    else:
        return 0
    
#Creating new column SexIdentHomosexual via applying function Homosexual
df['SexIdentHomosexual'] = df['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'].apply(lambda x:Homosexual (x))

In [54]:
#Creating function for column SexIdentBiSexual
def Bisexual (x):
    if x == 'Bisexual':
        return 1
    elif x == 'REFUSED':
        return -7
    elif x == 'NaN':
        return -9
    else:
        return 0
    
#Creating new column SexIdentBiSexual via applying function Bisexual
df['SexIdentBisexual'] = df['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'].apply(lambda x:Bisexual (x))

In [55]:
#Creating function for column SexIdentQueerPanaQuest
def Queer_Pan_Quest (x):
    if x == 'Queer, Pansexual, And/Or Questioning':
        return 1
    elif x == 'REFUSED':
        return -7
    elif x == 'NaN':
        return -9
    else:
        return 0
    
#Creating new column SexIdentQueerPanaQuest via applying function Queer_Pan_Quest
df['SexIdentQueerPanaQuest'] = df['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'].apply(lambda x:Queer_Pan_Quest (x))

In [56]:
#Creating function for column SexIdentAsexual
def Asexual (x):
    if x == 'Asexual':
        return 1
    elif x == 'REFUSED':
        return -7
    elif x == 'NaN':
        return -9
    else:
        return 0
    
#Creating new column SexIdentAsexual via applying function Asexual
df['SexIdentAsexual'] = df['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'].apply(lambda x:Asexual (x))

In [57]:
#Creating function for column SexIdentOther
def Other (x):
    if x == 'Other':
        return 1
    elif x == 'REFUSED':
        return -7
    elif x == 'NaN':
        return -9
    else:
        return 0
    
#Creating new column SexIdentHeterosexual via applying function Asexual
df['SexIdentOther'] = df['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'].apply(lambda x:Other (x))

In [58]:
#Creating function for column SexIdentSpec
def Other_Spec (x):
    if x == 'Other (SPECIFY)':
        return 1
    elif x == 'REFUSED':
        return -7
    elif x == 'NaN':
        return -9
    else:
        return 0
    
#Creating new column SexIdentSpec via applying function Other_Spec
df['SexIdentSpec'] = df['WHAT_DO_YOU_THINK_OF_YOURSELF_AS'].apply(lambda x:Other_Spec (x))

In [59]:
#Filling in null values with'NaN'
df['RELATIONSHIP_STATUS'] = df['RELATIONSHIP_STATUS'].fillna('NaN')

#Creating function for RelationshipStatus column
def Relationship_Status (x):
    if x == 'Married':
        return 1
    elif x == 'Single':
        return 2
    elif x == 'Divorced':
        return 3
    elif x == 'Separated':
        return 4
    elif x == 'Widowed':
        return 5
    elif x == 'In a relationship':
        return 6
    elif x == 'In multiple relationships':
        return 7
    elif x == 'REFUSED':
        return -7
    elif x == 'NaN':
        return -9
    
#Creating new column RelationshipStatus via applying function Relationship_Statu
df['RelationshipStatus'] = df['RELATIONSHIP_STATUS'].apply(lambda x:Relationship_Status (x))

In [60]:
#Filling missing values with 'NaN'
df['CURRENTLY_PREGNANT'] = df['CURRENTLY_PREGNANT'].fillna('NaN')

#Creating function for Pregnant column
def Pregnant (x):
    if x == 'Yes':
        return 1
    elif x == 'No':
        return 0
    elif x == 'Do not know':
        return -8
    elif x == 'REFUSED':
        return -7
    elif x == 'NaN':
        return -9
    
#Creating new column Pregnant via applying function Pregnant
df['Pregnant'] = df['CURRENTLY_PREGNANT'].apply(lambda x: Pregnant (x))

In [61]:
#Replacing null values with 'NaN'
df['HAVE_CHILDREN'] = df['HAVE_CHILDREN'].fillna(-9)

# Creating function to create new column Children
def Child (row):
    if row['HAVE_CHILDREN'] == 'Yes':
        return 1
    elif row['HAVE_CHILDREN'] == 'No (Skip to Question 9)':
        return 0
    elif row['HAVE_CHILDREN'] == 'REFUSED (Skip to Question 9)':
        return -7
    elif row['HAVE_CHILDREN'] == -9:
        return -9
    
def Child_code(row):
    if row['InterviewType']==1:
        return Child(row)
    else:
        pass
    
df['Children'] = df.apply(Child_code,axis=1)

In [62]:
#Renaming columns 
df.rename(columns = {'CHILDREN_UNDER_18':'ChildrenUnder18Nr'}, inplace = True)


def ChildrenUnder18Nr(row):
    if row['Children'] == 1:
        return row['ChildrenUnder18Nr']
    else:
        pass
    
df['ChildrenUnder18Nr'] = df.apply(ChildrenUnder18Nr,axis=1)

In [63]:
#Creating function to create column ChildrenCustody
def custody(row):
    if row['CHILDREN_UNDER_18_LIVING_SOMEWHERE_ELSE_COURT_INTERVENTION'] == 'Yes':
        return 1
    elif  row['CHILDREN_UNDER_18_LIVING_SOMEWHERE_ELSE_COURT_INTERVENTION'] == 'No (Skip to Question 9)':
        return 0

def Children_Custody_code(row):
    if row['Children'] != 1:
        pass
    elif row['ChildrenUnder18Nr'] <= 0:
        pass
    elif row['ChildrenUnder18Nr'] > 0:
        return custody(row)
    
df['ChildrenCustody'] = df.apply(Children_Custody_code,axis=1)

In [64]:
#Renaming column 
df.rename(columns = {'CHILDREN_REMOVED_FROM_CLIENTS_CARE':'ChildrenCustodyNr'}, inplace = True)

def custody_nr (row):
    if row['ChildrenCustody'] == 1:
        return row['ChildrenCustodyNr'] 

def ChildrenCustodyNr(row):
    if row['Children'] != 1:
        pass
    elif row['ChildrenUnder18Nr'] <= 0:
        pass
    else:
        return custody_nr(row)

df['ChildrenCustodyNr'] = df.apply(ChildrenCustodyNr,axis=1)

In [65]:
#Creating function to create column ChildrenReunited
def child_reunited(row):
    if row ['REUNITED_WITH_CHILDREN_UNDER_18_PREV_REMOVED_FROM_CARE'] == 'Yes':
        return 1
    elif row ['REUNITED_WITH_CHILDREN_UNDER_18_PREV_REMOVED_FROM_CARE']== 'No':
        return 0

def Children_Reunited_code(row):
    if row['Children'] != 1:
        pass
    elif row['ChildrenUnder18Nr'] <= 0:
        pass
    elif row['ChildrenCustody'] != 1:
        pass
    else:
        return child_reunited(row)
    
df['ChildrenReunited'] = df.apply(Children_Reunited_code,axis=1)

In [66]:
#Renaming column 
df.rename(columns = {'CHILDREN_REUNITED_WITH_CLIENT':'ChildrenReunitedNr'},inplace = True)

def ChildrenReunitedNr(row):
    if row['ChildrenReunited'] == 1:
        return row['ChildrenReunitedNr']
    
df['ChildrenReunitedNr'] = df.apply(ChildrenReunitedNr,axis=1)

In [67]:
#Replacing null values 'NaN'
df['EVER_SERVED_IN_MILITARY'] = df['EVER_SERVED_IN_MILITARY'].fillna('NaN')

#Creating function to create new column MilitaryServed
def Military_Served (x):
    if x == 'No':
        return 0
    elif x == 'Yes, In the Armed Forces':
        return 1
    elif x == 'Yes, In the National Guard':
        return 3
    elif x == 'REFUSED':
        return -7
    elif x == 'NaN':
        return -9

# Creating new column MilitaryServed applying function Military_Served
df['MilitaryServed'] = df['EVER_SERVED_IN_MILITARY'].apply(lambda x:Military_Served (x))

In [68]:
#Replacing null values 'NaN'
df['HOW_LONG_TO_TRAVEL_TO_GRANT_SERVICES_LOCATION'] = df['HOW_LONG_TO_TRAVEL_TO_GRANT_SERVICES_LOCATION'].fillna('NaN')

#Creating function to create new column ServicesTravelTime
def Travel_Time (x):
    if x == 'Half an hour or less':
        return 1
    elif x == 'Between half an hour and one hour':
        return 2
    elif x == 'Between one hour and one and a half hours':
        return 3
    elif x == 'Between one and a half hours and two hours':
        return 4
    elif x == 'Two hours or more':
        return 5
    elif x == 'REFUSED':
        return -7
    elif x == 'NaN':
        return -9

#Creating new column ServicesTravelTime via applying function Travel_Time
df['ServicesTravelTime'] = df['HOW_LONG_TO_TRAVEL_TO_GRANT_SERVICES_LOCATION'].apply(lambda x: Travel_Time (x))

In [69]:
#Renaming column
df.rename(columns = {'ALCOHOL_NUMBER_OF_DAYS':'AlcoholDays'}, inplace = True)

#Replacing null values with 0 to create column AlcoholDays
df['AlcoholDays'] = df['AlcoholDays'].fillna(0)

#Replacing values greater than 30 with 30
df['AlcoholDays'] = df['AlcoholDays'].apply(lambda x:exceed_thirty_days(x))


In [70]:
#Creating function for AlcoholRoute
def Alcohol_Route (row):
    if row['ALCOHOL_ROUTE'] == '1. Oral':
        return 1
    elif row['AlcoholDays'] ==0:
        return -1
    else:
        return -1 
    
#Creating column AlcoholRoute via applying the function Alcohol_Route
df['AlcoholRoute'] = df.apply(Alcohol_Route, axis = 1 )

In [71]:
df['ALCOHOL_ROUTE'].unique()

array([nan, '1. Oral'], dtype=object)

In [72]:
#Creating column AlcoholOther1Days 
df['AlcoholOther1Days'] = 0

#Creating column AlcoholOther1Route 
df['AlcoholOther1Route'] = -1

df['AlcoholOther1Spec'] = " "

#Replacing missing values to create respective columns
df['AlcoholOther2Days'] = 0
df['AlcoholOther2Route'] = -1
df['AlcoholOther2Spec'] = " "
df['AlcoholOther3Days'] = 0
df['AlcoholOther3Route'] = -1
df['AlcoholOther3Spec'] = " "

In [73]:
#Renaming colulmn
df.rename(columns = {'HEROIN_NUMBER_OF_DAYS':'OpioidsHeroinDays'}, inplace = True)

#Replacing null values with 0 to create column OpioidsHeroinDays
df['OpioidsHeroinDays'] = df['OpioidsHeroinDays'].fillna(0)

df['OpioidsHeroinDays'] = df['OpioidsHeroinDays'].apply(lambda x:exceed_thirty_days(x))

In [74]:
#Creating substrings
df['HEROIN_ROUTE'] = df['HEROIN_ROUTE'].str.split(',').str[0]

#Creating function for column OpioidsHeroinRoute
def Heroin_Route (row):
    if row['HEROIN_ROUTE'] == '1. Oral':
        return 1
    elif row['HEROIN_ROUTE'] == '2. Intranasal':
        return 2
    elif row['HEROIN_ROUTE'] == '5. Non-IV Injection':
        return 4
    elif row['HEROIN_ROUTE'] == '6. Intravenous (IV) Injection':
        return 5
    elif row['HEROIN_ROUTE'] == '0. Other':
        return 0
    elif row['HEROIN_ROUTE'] == '4. Smoking':
        return 3
    elif row['OpioidsHeroinDays'] == 0:
        return -1
    else:
        return -1
    
#Creating column OpioidsHeroinRoute via applying function Heroin_Route
df['OpioidsHeroinRoute'] = df.apply(Heroin_Route, axis = 1)

In [75]:
#Renaming columns
df.rename(columns = {'MORPHINE_NUMBER_OF_DAYS':'OpioidsMorphineDays'}, inplace = True)

# Amending 'MORPHINE_NUMBER_OF_DAYS' column via replacing null values with 0 to create OpioidsMorphineDays column
df['OpioidsMorphineDays'] = df['OpioidsMorphineDays'].fillna(0)

df['OpioidsMorphineDays'] = df['OpioidsMorphineDays'].apply(lambda x: exceed_thirty_days(x))

In [76]:
#Creating function to create new column OpioidsMorphineRoute
def Morphine_Route (row):
    if row['MORPHINE_ROUTE'] == '1. Oral':
        return 1
    elif row['MORPHINE_ROUTE'] == '6. Intravenous (IV) Injection':
        return 2
    elif row['MORPHINE_ROUTE'] == '0. Other':
        return 0
    elif row['OpioidsMorphineDays'] == 0:
        return -1
    
#Creating new column OpioidsMorphineRoute via applying the function Morphine_Route
df['OpioidsMorphineRoute'] = df.apply(Morphine_Route, axis = 1)

In [77]:
#Renaming columns 
df.rename(columns = {'FENTANYL_NUMBER_OF_DAYS':'OpioidsFentanylDays'},inplace = True)

# Replacing null values with 0 to create OpioidsFentanylDays column 
df['OpioidsFentanylDays'] = df['OpioidsFentanylDays'].fillna(0)

df['OpioidsFentanylDays'] = df['OpioidsFentanylDays'].apply(lambda x:exceed_thirty_days(x))

In [78]:
#Creating function to create new column OpioidsFentanylRoute
def Fentanyl_Route (row):
    if row['FENTANYL_ROUTE'] == '1. Oral':
        return 1
    elif row['FENTANYL_ROUTE'] == '2. Intranasal':
        return 2
    elif row['FENTANYL_ROUTE'] == '6. Intravenous (IV) Injection':
        return 5
    elif row['FENTANYL_ROUTE'] == '0. Other':
        return 0
    elif row['OpioidsFentanylDays']==0:
        return -1
    else:
        return -1
    
#Creating new column OpioidsFentanylRoute via applying the function Fentanyl_Route
df['OpioidsFentanylRoute'] = df.apply(Fentanyl_Route, axis = 1)

In [79]:
#Renaming column
df.rename(columns = {'DILAUDID_NUMBER_OF_DAYS':'OpioidsDilaudidDays'}, inplace = True)

#Replacing null values with 0 to create values for column OpioidsDilaudidDays
df['OpioidsDilaudidDays'] = df['OpioidsDilaudidDays'].fillna(0)

df['OpioidsDilaudidDays'] = df['OpioidsDilaudidDays'].apply(lambda x:exceed_thirty_days(x))

In [80]:
#Creating function for creating column OpioidsDilaudidRoute
def Dilaudid_Route (row):
    if row ['DILAUDID_ROUTE'] == '1. Oral':
        return 1
    elif row['OpioidsDilaudidDays'] == 0:
        return -1
    else:
        return -1

# Creating new column OpioidsDilaudidRoutev via applying function Dilaudid_Route  
df['OpioidsDilaudidRoute'] = df.apply(Dilaudid_Route,axis = 1)

In [81]:
#Renaming column
df.rename(columns = {'DEMEROL_NUMBER_OF_DAYS':'OpioidsDemerolDays'}, inplace = True)

#Replacing nulll values with 0 to create values for column OpioidsDemerolDays
df['OpioidsDemerolDays'] = df['OpioidsDemerolDays'].fillna(0)

df['OpioidsDemerolDays'] = df['OpioidsDemerolDays'].apply(lambda x: exceed_thirty_days(x))


## Exporting GPRA1


In [83]:
# #Create variable by indexing columns and creating df
# GPRA1 = df[['MRN','ClientID','ClientDescripTreat','ClientDescripRecov','GrantNo',	'InterviewType',	'ConductedInterview',	'InterviewDate',	'IntakeSeqNum',	'GrantInactFlag',	'InactFlag',	'FFY',	'Quarter',	'Month',	'BirthMonth',	'BirthYear',	'Age',	'AgeGroup',	'Gender',	'GenderSpec',	'HispanicLatino',	'EthnicCentralAmerican',	'EthnicCuban',	'EthnicDominican',	'EthnicMexican',	'EthnicPuertoRican',	'EthnicSouthAmerican',	'EthnicOther',	'EthnicOtherSpec',	'RaceBlack',	'RaceWhite',	'RaceAmericanIndian',	'RaceAlaskaNative',	'RaceAsianIndian',	'RaceChinese',	'RaceFilipino',	'RaceJapanese',	'RaceKorean',	'RaceVietnamese',	'RaceOtherAsian',	'RaceNativeHawaiian',	'RaceGuamanianChamorro',	'RaceSamoan',	'RaceOtherPacificIslander',	'RaceOther',	'RaceSpec',	'LangNotEnglishAtHome',	'LangNotEnglishSpoken',	'LangNotEnglishSpokenSpec',	'LangNotSpanishAtHome',	'LangNotSpanishSpoken',	'LangNotSpanishSpokenSpec',	'SexIdentHeterosexual',	'SexIdentHomosexual',	'SexIdentBisexual',	'SexIdentQueerPanaQuest',	'SexIdentAsexual',	'SexIdentOther',	'SexIdentSpec',	'RelationshipStatus',	'Pregnant',	'Children',	'ChildrenUnder18Nr',	'ChildrenCustody',	'ChildrenCustodyNr',	'ChildrenReunited',	'ChildrenReunitedNr',	'MilitaryServed',	'ServicesTravelTime',	'AlcoholDays',	'AlcoholRoute',	'AlcoholOther1Days',	'AlcoholOther1Route',	'AlcoholOther1Spec',	'AlcoholOther2Days',	'AlcoholOther2Route',	'AlcoholOther2Spec',	'AlcoholOther3Days',	'AlcoholOther3Route',	'AlcoholOther3Spec',	'OpioidsHeroinDays',	'OpioidsHeroinRoute',	'OpioidsMorphineDays',	'OpioidsMorphineRoute',	'OpioidsFentanylDays',	'OpioidsFentanylRoute',	'OpioidsDilaudidDays',	'OpioidsDilaudidRoute',	'OpioidsDemerolDays']]

# #Exporting GPRA 1
# GPRA1.to_csv('GPRA1.csv', index = False, header = True, encoding = 'utf-8')

#Create variable by indexing columns and creating df
GPRA1_STAR = df[['BatchID','PATIENT','PROVIDER_COUNTY','MRN','ClientID','ClientDescripTreat','ClientDescripRecov','GrantNo',	'InterviewType',	'ConductedInterview',	'InterviewDate',	'IntakeSeqNum',	'GrantInactFlag',	'InactFlag',	'FFY',	'Quarter',	'Month',	'BirthMonth',	'BirthYear',	'Age',	'AgeGroup',	'Gender',	'GenderSpec',	'HispanicLatino',	'EthnicCentralAmerican',	'EthnicCuban',	'EthnicDominican',	'EthnicMexican',	'EthnicPuertoRican',	'EthnicSouthAmerican',	'EthnicOther',	'EthnicOtherSpec',	'RaceBlack',	'RaceWhite',	'RaceAmericanIndian',	'RaceAlaskaNative',	'RaceAsianIndian',	'RaceChinese',	'RaceFilipino',	'RaceJapanese',	'RaceKorean',	'RaceVietnamese',	'RaceOtherAsian',	'RaceNativeHawaiian',	'RaceGuamanianChamorro',	'RaceSamoan',	'RaceOtherPacificIslander',	'RaceOther',	'RaceSpec',	'LangNotEnglishAtHome',	'LangNotEnglishSpoken',	'LangNotEnglishSpokenSpec',	'LangNotSpanishAtHome',	'LangNotSpanishSpoken',	'LangNotSpanishSpokenSpec',	'SexIdentHeterosexual',	'SexIdentHomosexual',	'SexIdentBisexual',	'SexIdentQueerPanaQuest',	'SexIdentAsexual',	'SexIdentOther',	'SexIdentSpec',	'RelationshipStatus',	'Pregnant',	'Children',	'ChildrenUnder18Nr',	'ChildrenCustody',	'ChildrenCustodyNr',	'ChildrenReunited',	'ChildrenReunitedNr',	'MilitaryServed',	'ServicesTravelTime',	'AlcoholDays',	'AlcoholRoute',	'AlcoholOther1Days',	'AlcoholOther1Route',	'AlcoholOther1Spec',	'AlcoholOther2Days',	'AlcoholOther2Route',	'AlcoholOther2Spec',	'AlcoholOther3Days',	'AlcoholOther3Route',	'AlcoholOther3Spec',	'OpioidsHeroinDays',	'OpioidsHeroinRoute',	'OpioidsMorphineDays',	'OpioidsMorphineRoute',	'OpioidsFentanylDays',	'OpioidsFentanylRoute',	'OpioidsDilaudidDays',	'OpioidsDilaudidRoute',	'OpioidsDemerolDays']]

#Exporting GPRA 1
GPRA1_STAR.to_csv('GPRA1_STAR.csv', index = False, header = True, encoding = 'utf-8')